In [ ]:
# This file generates incorrect answers for TriviaQA questions and saves a version of the TriviaQA dataset with incorrect alternative answer choices.

In [23]:
from datasets import load_dataset, load_from_disk
import os
import random
from tqdm import tqdm
import string
import json
import openai
from utilities import get_gpt3_outputs

In [24]:
trivia_qa = load_from_disk("trivia_qa")

In [25]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [27]:
def generate_prompt(question, answer):
    baseline_prompt = """Who was President when the first Peanuts cartoon was published?
A: Harry Truman
B: George Bush
C: Dwight Eisenhower
D: Franklin Roosevelt

Which American-born Sinclair won the Nobel Prize for Literature in 1930?
A: Sinclair Lewis
B: Upton Sinclair
C: John Sinclair
D: Harry Sinclair

In which decade did Billboard magazine first publish and American hit chart?
A: 30s
B: 40s
C: 50s
D: 60s

Who won Super Bowl XX?
A: Chicago Bears
B: Miami Dolphins
C: New York Giants
D: San Francisco 49ers

What first appeared in Ohio in 1914 to affect transport?
A: First Traffic Lights
B: First Highway
C: First Parking Meter
D: First Train

"""
    return baseline_prompt + question + "\nA: " + answer + "\nB:"

In [29]:
trivia_qa["train"] = trivia_qa["train"].select(random.sample(range(len(trivia_qa["train"])), 10000))

In [30]:
prompts = []
for data in tqdm(trivia_qa["train"]):
    prompts.append(generate_prompt(data["question"], data["answer"]["value"]))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:05<00:00, 1821.22it/s]


In [32]:
outputs = get_gpt3_outputs(prompts)

In [33]:
processed_outputs = []
for output in outputs:
    processed_outputs.append(output.strip(" ").replace("C: ", "").replace("D: ", "").split("\n")[:3])

[['Ernest Shackleton', 'David Livingstone', 'Frankincense'], ['Lamborghini', 'Maserati', 'Alfa Romeo'], ['Stan Shunpike', 'Errol', 'Harry Potter'], ['House', 'Prison Break', 'Lost'], ['(Structure of) RNA', '(Structure of) proteins', '(Structure of) cell membranes'], ['KNEE bone', 'SHOULDER bone', 'HIP bone'], ['CAIRO', 'ROME', 'MADRID'], ['Fish', 'Plants', 'Animals'], ['France', 'Italy', 'Japan'], ['Jimmy Carter', 'Gerald Ford', 'Richard Nixon'], ['Sussex', 'Essex', 'Hampshire'], ['Shoulders', 'Hands', 'Feet'], ['Georgia', 'North Carolina', 'South Carolina'], ['Sweden', 'Norway', 'Denmark'], ['D', 'G', 'H'], ['James Joyce', 'William Faulkner', 'Ernest Hemingway'], ['Bay of Pigs', 'Bay of Plenty', 'Bay of Fundy'], ['Jihad', 'Ramada', 'Salat'], ['Columbine', 'Daffodil', 'Snapdragon'], ['Baghdad', 'Kabul', 'Islamabad'], ['Cubism', 'Surrealism', 'Expressionism'], ['Nigeria', 'Ghana', 'Kenya'], ['Uterus', 'Uranium', 'Utensil'], ['PARIS', 'BERLIN', 'WARSAW'], ['Dallas', 'Houston', 'San Anton

In [34]:
trivia_qa["train"] = trivia_qa["train"].add_column("wrong_answers", processed_outputs)

Flattening the indices:   0%|          | 0/10 [00:00<?, ?ba/s]

In [35]:
trivia_qa.save_to_disk("trivia_qa_wrong_answers_easy")